In [87]:
%matplotlib inline
import pprint  
import math
import numpy as np
import numpy.random as npr
import pandas as pd
import random
from scipy import stats
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from bs4 import Comment
import requests
import mechanize
import csv
import time
import re

In [88]:
College_Stats = pd.DataFrame(columns=('Player Name', 'Player ID','G','MP','FG','FGA','3P','3PA','FT','FTA','ORB',
                                      'TRB','AST','STL','BLK','TOV','PF','PTS','Shooting FG%','Shooting 3P%',
                                      'Shooting FT%','Per Game MP','Per Game PTS','Per Game TRB','Per Game AST'))

In [89]:
def url_change(playerID):
    url = 'http://www.basketball-reference.com/players/'+playerID[0]+'/'+playerID+'.html'
    return url

####Browser mechanize functionality
#br = mechanize.Browser(factory=mechanize.RobustFactory())
br = mechanize.Browser()
# cartype = 'Rolls-Royce'
# url = 'http://www.everycarlisted.com/search/vehicle_condition-New/make-'+cartype+'/year-2017-2017/page-'
# br.open(url+str(1))
# response = br.response()
# soup = BeautifulSoup(response,"html.parser")

def Open_Browser(url):
    br.open(url)
    response = br.response()
    soup = BeautifulSoup(response,"html.parser")
    return soup

def web2var(string):
    string = string.replace('+','')
    string = string.replace('-','')
    return string

def var2web(string):
    if ' ' in string:
        string = string.replace(' ','+')
    return string

def getMaxVORP(playerID):
    stringMatch = 'data-stat="vorp"'
    regex = '[+-]?\d+(?:\.\d+)?'
    url = url_change(playerID)
    soup = Open_Browser(url)
    soup = soup.find('div',{"id":"all_advanced"})
    if soup == None:
        return None
    soup_txt = str(soup.contents)
    vorp = []
    for m in re.finditer(stringMatch, soup_txt):
        num = re.findall(regex, soup_txt[m.start()+len(stringMatch)+2:m.end()+6])
        vorp.append(num)
    num_vorp = 0
    for i in range(0,len(vorp)):
        if not not vorp[i]:
            num_vorp = num_vorp + 1
        else:
            num_vorp = num_vorp
    vorp = max(vorp)
    return vorp, num_vorp

def getCollegeStats(playerID, playerName):
    stringMatch = ['data-stat="g"','data-stat="mp"','data-stat="fg"','data-stat="fga"','data-stat="fg3"','data-stat="fg3a"',
               'data-stat="ft"','data-stat="fta"','data-stat="orb"','data-stat="trb"','data-stat="ast"','data-stat="stl"',
               'data-stat="blk"','data-stat="tov"','data-stat="pf"','data-stat="pts"','data-stat="fg_pct"','data-stat="fg3_pct"',
               'data-stat="ft_pct"','data-stat="mp_per_g"','data-stat="pts_per_g"','data-stat="trb_per_g"','data-stat="ast_per_g"']
    regex = '[+-]?\d+(?:\.\d+)?'
    url = url_change(playerID)
    soup = Open_Browser(url)
    soup = soup.find('div',{"id":"all_all_college_stats"})
    if soup == None:
        row = [None] * 23
        return row
    comments=soup.find(string=lambda text:isinstance(text,Comment))
    comments = comments[comments.find("tfoot"):]
    row = [] 
    for i in range(0,len(stringMatch)): 
        for m in re.finditer(stringMatch[i], str(comments)): 
            num = re.findall(regex, comments[m.start()+len(stringMatch[i])+1:m.end()+6]) 
            row.append(num)
    if len(row) == 23:
        return row

In [90]:
df = pd.read_csv('NBADraft 2016-1985.csv')

In [91]:
Player_Name = df['Round 1 Player'] 
Player_ID = df['Player ID']

In [92]:
isinstance(Player_ID[356],basestring)
# Player_ID[350]

False

In [93]:
Player_ID[410]

nan

In [94]:
###College Stat Scrape
for k in range(0,len(Player_Name)-1):
    if isinstance(Player_ID[k],basestring):
        row = getCollegeStats(playerID=Player_ID[k], playerName= Player_Name[k])
        College_Stats = College_Stats.append([{'Player Name':Player_Name[k], 'Player ID':Player_ID[k],'G':row[0],'MP':row[1],'FG':row[2]
                                       ,'FGA':row[3],'3P':row[4],'3PA':row[5],'FT':row[6],'FTA':row[7],'ORB':row[8],
                                       'TRB':row[9],'AST':row[10],'STL':row[11],'BLK':row[12],'TOV':row[13],'PF':row[14]
                                       ,'PTS':row[15],'Shooting FG%':row[16],'Shooting 3P%':row[17],'Shooting FT%':row[18],
                                       'Per Game MP':row[19],'Per Game PTS':row[20],'Per Game TRB':row[21],'Per Game AST':row[22]}])        
        print k
    else:
        print('missing',k)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [98]:
#Write College Stat to CSV
College_Stats.to_csv('NBADraft CollegeData Scrape.csv', sep = ',', encoding = 'utf-8' )

In [ ]:
#VORP Scrape only
VORP = []
Num_VORP = []
count = 0 
for i in range(0,len(df)):
    count = count+1
    if isinstance(df['Player ID'][i],basestring):
        print count
        if getMaxVORP(df['Player ID'][i])== None:
            VORP.append(None)
            Num_VORP.append(None)            
        else:
            val = getMaxVORP(df['Player ID'][i])
            VORP.append(val[0])
            Num_VORP.append(val[1])
    else:
        VORP.append(None)
        Num_VORP.append(None)        
        print count
        

In [286]:
len(VORP)

2181

In [307]:
se = pd.Series(VORP)
se2 = pd.Series(Num_VORP)

In [305]:
df['NBA VORP'] = se.values

/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [308]:
df['NBA VORP Count'] = se2.values

/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [309]:
df.head()

Year  Rk  Pk   Tm            Round 1 Player  Player ID  \
0  2016   1   1  PHI     Ben Simmons\simmobe01  simmobe01   
1  2016   2   2  LAL  Brandon Ingram\ingrabr01  ingrabr01   
2  2016   3   3  BOS    Jaylen Brown\brownja02  brownja02   
3  2016   4   4  PHO   Dragan Bender\bendedr01  bendedr01   
4  2016   5   5  MIN        Kris Dunn\dunnkr01   dunnkr01   

              Round 1 College  Yrs  Totals G  Totals MP       ...        \
0  Louisiana State University  NaN       NaN        NaN       ...         
1             Duke University  1.0      79.0     2279.0       ...         
2    University of California  1.0      78.0     1341.0       ...         
3                         NaN  1.0      43.0      574.0       ...         
4          Providence College  1.0      78.0     1333.0       ...         

   Per Game MP  Per Game PTS  Per Game TRB  Per Game AST  Advanced WS  \
0          NaN           NaN           NaN           NaN          NaN   
1         28.8           9.4           4.0           2.1         -0.3   
2         17.2           6.6           2.8           0.8          1.5   
3         13.3           3.4           2.4           0.5         -0.3   
4         17.1           3.8           2.1           2.4          0.1   

   Advanced WS/48  Advanced BPM  Advanced VORP  NBA VORP  NBA VORP Count  
0             NaN           NaN            NaN      None             NaN  
1          -0.007          -3.8           -1.1    [-1.1]             2.0  
2           0.053          -4.0           -0.7    [-0.7]             2.0  
3          -0.029          -4.3           -0.3    [-0.3]             2.0  
4           0.004          -2.2           -0.1    [-0.1]             2.0  

[5 rows x 26 columns]

In [311]:
df.to_csv('C:\Users\gordon.tsai\Documents\NBADraft\NBADraft 1985-2016 Scrape.csv', sep = ',', encoding = 'utf-8' )